In [3]:
from z3 import *

def compute_tax_fixed(big_category, sub_category, quantity):
    """
    計算固定稅額類別的應納稅額（針對水泥及油氣類）
    
    參數:
      big_category: 字串, "cement" 或 "oilgas"
      sub_category: 字串
         若 big_category == "cement":
           "white"         -> 白水泥或有色水泥 (600 NT$/ton)  (貨物稅條例 第7條)
           "bute1"         -> 卜特蘭一型 (320 NT$/ton)         (貨物稅條例 第7條)
           "bute2"         -> 卜特蘭高爐 (196 NT$/ton)         (原法條為280元，經調整)
           "other_cement"  -> 代水泥及其他水泥 (440 NT$/ton)    (貨物稅條例 第7條)
         若 big_category == "oilgas":
           "gasoline"      -> 汽油 (6830 NT$/unit)             (貨物稅條例 第10條)
           "diesel"        -> 柴油 (3990 NT$/unit)             (貨物稅條例 第10條)
           "kerosene"      -> 煤油 (4250 NT$/unit)             (貨物稅條例 第10條)
           "aviation"      -> 航空燃油 (610 NT$/unit)          (貨物稅條例 第10條)
           "fuel_oil"      -> 燃料油 (110 NT$/unit)            (貨物稅條例 第10條)
           "solvent_oil"   -> 溶劑油 (720 NT$/unit)            (貨物稅條例 第10條)
           "lpg"           -> 液化石油氣 (690 NT$/ton)         (貨物稅條例 第10條)
      quantity: 當月份出廠應稅數量 (整數)
      
    計算公式:
      應納稅額 = (單位稅額) * (數量)
      
    回傳:
      Z3 求解後的應納稅金額
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    # 根據大種類決定 mapping，mapping 中之數值依據各法條規定
    if big_category == "cement":
        mapping = {
            "white": 600,         # 白水泥或有色水泥，每公噸600元 (貨物稅條例 第7條)
            "bute1": 320,          # 卜特蘭一型，每公噸320元 (貨物稅條例 第7條)
            "bute2": 196,          # 卜特蘭高爐，每公噸196元 (原法條為280元，經調整)
            "other_cement": 440    # 代水泥及其他水泥，每公噸440元 (貨物稅條例 第7條)
        }
    elif big_category == "oilgas":
        mapping = {
            "gasoline": 6830,      # 汽油，每公秉6830元 (貨物稅條例 第10條)
            "diesel": 3990,        # 柴油，每公秉3990元 (貨物稅條例 第10條)
            "kerosene": 4250,      # 煤油，每公秉4250元 (貨物稅條例 第10條)
            "aviation": 610,       # 航空燃油，每公秉610元 (貨物稅條例 第10條)
            "fuel_oil": 110,       # 燃料油，每公秉110元 (貨物稅條例 第10條)
            "solvent_oil": 720,    # 溶劑油，每公秉720元 (貨物稅條例 第10條)
            "lpg": 690             # 液化石油氣，每公噸690元 (貨物稅條例 第10條)
        }
    else:
        mapping = {}
    
    unit_tax = mapping.get(sub_category, 0)
    # 加入 constraint: tax_amount = unit_tax * quantity
    s.add(tax_amount == unit_tax * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

def compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric=False):
    """
    計算從價稅類別的應納稅額（針對橡膠輪胎、飲料品、平板玻璃、電器、車輛）
    
    參數:
      big_category: 字串，必須為下列之一：
           "rubber"    (橡膠輪胎)
           "beverage"  (飲料品)
           "flat_glass" (平板玻璃)
           "electrical" (電器)
           "vehicle"   (車輛)
      sub_category: 字串，依據大類別定義：
         若 big_category == "rubber":
           "bus_truck" -> 大客車、大貨車使用者 (從價徵收10%)
           "other"     -> 其他 (從價徵收15%)
           "exempt"    -> 免稅 (0%)
         若 big_category == "beverage":
           "diluted_juice" -> 稀釋天然果蔬汁 (從價徵收8%)
           "other"         -> 其他飲料品 (從價徵收15%)
           "exempt"        -> 免稅 (0%)
         若 big_category == "flat_glass":
           "regular"   -> 一般 (從價徵收10%)
           "exempt"    -> 免稅 (0%)
         若 big_category == "electrical":
           "refrigerator"           -> 電冰箱 (從價徵收13%)
           "color_tv"               -> 彩色電視機 (從價徵收13%)
           "central_aircon"         -> 冷暖氣機(中央系統型) (從價徵收15%)
           "non_central_aircon"     -> 冷暖氣機(非中央系統型) (從價徵收20%)
           "dehumidifier"           -> 除濕機 (從價徵收15%)
           "videorecorder"          -> 錄影機 (從價徵收13%)
           "non_portable_recordplayer" -> 電唱機(非手提32公分) (從價徵收10%)
           "portable_recordplayer"  -> 電唱機(手提32公分以下) (免稅 0%)
           "tape_recorder"          -> 錄音機 (從價徵收10%)
           "stereo"                 -> 音響組合 (從價徵收10%)
           "electric_oven"          -> 電烤箱 (從價徵收15%)
         若 big_category == "vehicle":
           "compact_car"   -> 小客車(2000cc以下) (從價徵收25%)
           "large_car"     -> 小客車(2000cc以上) (從價徵收30%)
           "truck_bus_others" -> 貨車、大客車及其他 (從價徵收15%)
           "motorcycle"    -> 機車 (從價徵收17%)
      quantity: 當月份出廠應稅數量 (整數)
      assessed_price: 每單位完稅價格 (例如由銷售價格/(1+稅率) 得到)
      is_electric: (僅針對 vehicle 類) 布林值，若為 True 則稅率依規定減半
      
    計算公式:
      應納稅額 = assessed_price * (從價稅率) * quantity
      (若為車輛且 is_electric 為 True，則稅率折半)
      
    回傳:
      Z3 求解後的應納稅金額
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    # 根據大類別與子類別決定稅率
    if big_category == "rubber":
        mapping = {"bus_truck": 0.10, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "beverage":
        mapping = {"diluted_juice": 0.08, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "flat_glass":
        mapping = {"regular": 0.10, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "electrical":
        mapping = {
            "refrigerator": 0.13,
            "color_tv": 0.13,
            "central_aircon": 0.15,
            "non_central_aircon": 0.20,
            "dehumidifier": 0.15,
            "videorecorder": 0.13,
            "non_portable_recordplayer": 0.10,
            "portable_recordplayer": 0.0,
            "tape_recorder": 0.10,
            "stereo": 0.10,
            "electric_oven": 0.15
        }
        rate = mapping.get(sub_category, 0)
    elif big_category == "vehicle":
        mapping = {
            "compact_car": 0.25,
            "large_car": 0.30,
            "truck_bus_others": 0.15,
            "motorcycle": 0.17
        }
        rate = mapping.get(sub_category, 0)
        # 若為電動車或油電混合車輛，則稅率折半
        if is_electric:
            rate = rate / 2.0
    else:
        rate = 0
    
    # 加入 constraint: tax_amount = assessed_price * rate * quantity
    s.add(tax_amount == assessed_price * rate * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

# 測試範例
if __name__ == '__main__':
    # 範例1：水泥類，選擇白水泥，出廠數量10公噸
    tax1 = compute_tax_fixed("cement", "white", 10)
    print("水泥類 Tax:", tax1)  # 預期 600 * 10 = 6000
    
    # 範例2：油氣類，選擇汽油，出廠數量5 (單位：公秉)
    tax2 = compute_tax_fixed("oilgas", "gasoline", 5)
    print("油氣類 Tax:", tax2)  # 預期 6830 * 5 = 34150
    
    # 範例3：其他類目 - 電器，選擇冷暖氣機(中央系統型)，完稅價格2000元，出廠數量2台
    tax3 = compute_tax_others("electrical", "central_aircon", 2, 2000)
    print("電器 Tax:", tax3)  # 預期 2000 * 0.15 * 2 = 600
    
    # 範例4：其他類目 - 車輛，選擇小客車(2000cc以下) (25%)，非電動車，完稅價格800000元，出廠數量3輛
    tax4 = compute_tax_others("vehicle", "compact_car", 3, 800000, is_electric=False)
    print("車輛 Tax:", tax4)  # 預期 800000 * 0.25 * 3 = 600000
    
    # 範例5：其他類目 - 車輛，同上但若為則稅率減半 (0.25/2 = 0.125)
    tax5 = compute_tax_others("vehicle", "compact_car", 3, 800000, is_electric=True)
    print("電動車 Tax:", tax5)  # 預期 800000 * 0.125 * 3 = 300000


水泥類 Tax: 6000
油氣類 Tax: 34150
電器 Tax: 600
車輛 Tax: 600000
電動車 Tax: 300000


### Selenium

In [11]:
import random
import time
from decimal import Decimal
from z3 import Real, Solver, sat
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

# ------------------------------
# 計算器 function（固定稅額）
# ------------------------------
def compute_tax_fixed(big_category, sub_category, quantity):
    """
    計算固定稅額類別的應納稅額（針對水泥及油氣類）
    計算公式: 應納稅額 = (單位稅額) * (數量)
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    if big_category == "cement":
        mapping = {
            "white": 600,         # 白水泥或有色水泥，每公噸600元
            "bute1": 320,          # 卜特蘭一型
            "bute2": 196,          # 卜特蘭高爐
            "other_cement": 440    # 代水泥及其他水泥
        }
    elif big_category == "oilgas":
        mapping = {
            "gasoline": 6830,      # 汽油，每公秉6830元
            "diesel": 3990,        # 柴油，每公秉3990元
            "kerosene": 4250,      # 煤油，每公秉4250元
            "aviation": 610,       # 航空燃油，每公秉610元
            "fuel_oil": 110,       # 燃料油，每公秉110元
            "solvent_oil": 720,    # 溶劑油，每公秉720元
            "lpg": 690             # 液化石油氣，每公噸690元
        }
    else:
        mapping = {}
    
    unit_tax = mapping.get(sub_category, 0)
    s.add(tax_amount == unit_tax * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

# ------------------------------
# 計算器 function（從價稅）
# ------------------------------
def compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric=False):
    """
    計算從價稅類別的應納稅額（針對橡膠輪胎、飲料品、平板玻璃、電器、車輛）
    計算公式: 應納稅額 = assessed_price * (從價稅率) * quantity
      若為車輛且 is_electric 為 True，則稅率折半
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    if big_category == "rubber":
        mapping = {"bus_truck": 0.10, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "beverage":
        mapping = {"diluted_juice": 0.08, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "flat_glass":
        mapping = {"regular": 0.10, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "electrical":
        mapping = {
            "refrigerator": 0.13,
            "color_tv": 0.13,
            "central_aircon": 0.15,
            "non_central_aircon": 0.20,
            "dehumidifier": 0.15,
            "videorecorder": 0.13,
            "non_portable_recordplayer": 0.10,
            "portable_recordplayer": 0.0,
            "tape_recorder": 0.10,
            "stereo": 0.10,
            "electric_oven": 0.15
        }
        rate = mapping.get(sub_category, 0)
    elif big_category == "vehicle":
        mapping = {
            "compact_car": 0.25,
            "large_car": 0.30,
            "truck_bus_others": 0.15,
            "motorcycle": 0.17
        }
        rate = mapping.get(sub_category, 0)
        if is_electric:
            rate = rate / 2.0
    else:
        rate = 0
    
    s.add(tax_amount == assessed_price * rate * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

# ------------------------------
# 模擬逐字輸入函式
# ------------------------------
def human_type(element, text, delay=0.1):
    """
    模擬逐字輸入，依序將 text 中的字元輸入到 element，
    每個字元間隔 delay 秒，最後送出 TAB
    """
    for char in text:
        element.send_keys(char)
        time.sleep(delay)
    element.send_keys(Keys.TAB)

# ------------------------------
# Selenium 自動化測試主程式
# ------------------------------
def run_tax_calculator_test():
    # 啟動 Chrome WebDriver 並設定等待
    driver = webdriver.Chrome()  # 請確認 chromedriver 已加入 PATH
    wait = WebDriverWait(driver, 10)
    driver.delete_all_cookies()
    # 載入目標網頁
    driver.get("https://www.etax.nat.gov.tw/etwmain/etw158w/79")
    time.sleep(2)  # 初始等待讓網頁完全載入

    total_tests = 0
    mismatches = 0

    # 固定稅額部分對應設定（select1 與 select2）
    fixed_categories = {
        "cement": {
            "option_value": "1",   # 水泥
            "sub_options": {
                "white": "600",         # 白水泥或有色水泥，每公噸600元
                "bute1": "320",          # 卜特蘭一型
                "bute2": "196",          # 卜特蘭高爐
                "other_cement": "440"    # 代水泥及其他水泥
            }
        },
        "oilgas": {
            "option_value": "2",   # 油氣類
            "sub_options": {
                "gasoline": "6830",     # 汽油，每公秉6830元
                "diesel": "3990",       # 柴油，每公秉3990元
                "kerosene": "4250",     # 煤油，每公秉4250元
                "aviation": "610",      # 航空燃油，每公秉610元
                "fuel_oil": "110",      # 燃料油，每公秉110元
                "solvent_oil": "720",   # 溶劑油，每公秉720元
                "lpg": "690"            # 液化石油氣，每公噸690元
            }
        }
    }

    # 從價稅部分對應設定（select3 及 select4 / radio）
    others_categories = {
        "rubber": {
            "option_value": "1",  # 橡膠輪胎
            "sub_options": {
                "bus_truck": "10",   # 大客車、大貨車使用者，從價徵收10%
                "other": "15",       # 其他，各徵收15%
                "exempt": "0"        # 免稅
            }
        },
        "beverage": {
            "option_value": "2",  # 飲料品
            "sub_options": {
                "diluted_juice": "8",  # 稀釋天然果蔬汁，8%
                "other": "15",
                "exempt": "0"
            }
        },
        "flat_glass": {
            "option_value": "3",  # 平板玻璃
            "sub_options": {
                "regular": "10",   # 從價徵收10%
                "exempt": "0"      # 免稅
            }
        },
        "electrical": {
            "option_value": "4",  # 電器
            "sub_options": {
                "refrigerator": "13",           # 電冰箱 13%
                "color_tv": "13",               # 彩色電視機 13%
                "non_central_aircon": "20",     # 冷暖氣機 (非中央系統型) 20%
                "central_aircon": "15",         # 冷暖氣機 (中央系統型) 15%
                "dehumidifier": "15",           # 除濕機 15%
                "videorecorder": "13",          # 錄影機 13%
                "non_portable_recordplayer": "10",  # 電唱機 (非手提) 10%
                "portable_recordplayer": "0",   # 電唱機 (手提免稅)
                "tape_recorder": "10",          # 錄音機 10%
                "stereo": "10",                 # 音響組合 10%
                "electric_oven": "15"           # 電烤箱 15%
            }
        },
        "vehicle": {
            "option_value": "5",  # 車輛
            "radio_options": {
                "compact_car": "25",        # 小客車2000cc以下：25%
                "large_car": "30",          # 小客車2000cc以上：30%
                "truck_bus_others": "15",   # 貨車、大客車及其他：15%
                "motorcycle": "17"          # 機車：17%
            }
        }
    }

    # 測試迴圈：重複 100 次
    for i in range(100):
        total_tests += 1
        print(f"===== 測試案例 {i+1} =====")
        # 隨機決定測試類型：0 為固定稅額，1 為從價稅
        tax_type = random.choice([0, 1])
        if tax_type == 0:
            # 固定稅額測試
            big_category = random.choice(list(fixed_categories.keys()))
            sub_category = random.choice(list(fixed_categories[big_category]["sub_options"].keys()))
            quantity = random.randint(1, 50)
            local_result = compute_tax_fixed(big_category, sub_category, quantity)

            # ------------------------------
            # 固定稅額部分網頁操作
            # ------------------------------
            # 使用 Select 直接選擇下拉選單項目
            select1 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select1"))))
            select1.select_by_value(fixed_categories[big_category]["option_value"])
            time.sleep(0.5)
            select2 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select2"))))
            select2.select_by_value(fixed_categories[big_category]["sub_options"][sub_category])
            time.sleep(0.5)
            # 輸入單位稅額 (input#tax)
            tax_input = wait.until(EC.element_to_be_clickable((By.ID, "tax")))
            try:
                tax_input.clear()
            except Exception:
                driver.execute_script("arguments[0].value = '';", tax_input)
            human_type(tax_input, fixed_categories[big_category]["sub_options"][sub_category])
            time.sleep(0.5)
            # 輸入數量 (input#taxCount1)
            taxCount1 = wait.until(EC.element_to_be_clickable((By.ID, "taxCount1")))
            try:
                taxCount1.clear()
            except Exception:
                driver.execute_script("arguments[0].value = '';", taxCount1)
            human_type(taxCount1, str(quantity))
            time.sleep(0.5)
            # 按下第一個「計算」按鈕
            calc_buttons = driver.find_elements(By.XPATH, "//button[@title='計算']")
            calc_buttons[0].click()
            time.sleep(2)
            # 讀取結果 (第一個 span.text-danger)
            result_span = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.d-flex.align-items-center span.text-danger")))
            try:
                online_result = Decimal(result_span.text.strip())
            except Exception:
                online_result = None

            # 比對本地與網頁結果
            if online_result is None or Decimal(str(local_result)) != online_result:
                print("不一致測試案例 (固定稅額):")
                print(f"big_category: {big_category}, sub_category: {sub_category}, quantity: {quantity}")
                print(f"本地計算: {local_result}, 網頁結果: {online_result}")
                mismatches += 1

        else:
            # 從價稅測試
            big_category = random.choice(list(others_categories.keys()))
            if big_category == "vehicle":
                sub_category = random.choice(list(others_categories[big_category]["radio_options"].keys()))
            else:
                sub_category = random.choice(list(others_categories[big_category]["sub_options"].keys()))
            quantity = random.randint(1, 50)
            assessed_price = random.randint(100, 1000000)
            is_electric = False
            if big_category == "vehicle":
                is_electric = random.choice([True, False])
            local_result = compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric)

            # ------------------------------
            # 從價稅部分網頁操作
            # ------------------------------
            select3 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select3"))))
            select3.select_by_value(others_categories[big_category]["option_value"])
            time.sleep(0.5)
            if big_category == "vehicle":
                # 車輛使用 radio 按鈕
                radio_value = others_categories[big_category]["radio_options"][sub_category]
                radios = driver.find_elements(By.NAME, "radio1")
                for radio in radios:
                    if radio.get_attribute("value") == radio_value:
                        radio.click()
                        break
                time.sleep(0.5)
                # 處理電動車 checkbox (若存在)
                try:
                    checkbox = driver.find_element(By.ID, "carCheckBox")
                    if is_electric and not checkbox.is_selected():
                        checkbox.click()
                    elif not is_electric and checkbox.is_selected():
                        checkbox.click()
                except Exception:
                    print("未找到電動車 checkbox")
                time.sleep(0.5)
            else:
                # 其他類別使用 select4
                select4 = Select(wait.until(EC.element_to_be_clickable((By.ID, "select4"))))
                select4.select_by_value(others_categories[big_category]["sub_options"][sub_category])
                time.sleep(0.5)
                # 輸入稅率 (input#taxRate) － 以百分比表示
                mapping_rate = {}
                if big_category == "rubber":
                    mapping_rate = {"bus_truck": 10, "other": 15, "exempt": 0}
                elif big_category == "beverage":
                    mapping_rate = {"diluted_juice": 8, "other": 15, "exempt": 0}
                elif big_category == "flat_glass":
                    mapping_rate = {"regular": 10, "exempt": 0}
                elif big_category == "electrical":
                    mapping_rate = {
                        "refrigerator": 13,
                        "color_tv": 13,
                        "non_central_aircon": 20,
                        "central_aircon": 15,
                        "dehumidifier": 15,
                        "videorecorder": 13,
                        "non_portable_recordplayer": 10,
                        "portable_recordplayer": 0,
                        "tape_recorder": 10,
                        "stereo": 10,
                        "electric_oven": 15
                    }
                tax_rate_percent = mapping_rate.get(sub_category, 0)
                taxRate_input = wait.until(EC.element_to_be_clickable((By.ID, "taxRate")))
                try:
                    taxRate_input.clear()
                except Exception:
                    driver.execute_script("arguments[0].value = '';", taxRate_input)
                human_type(taxRate_input, str(tax_rate_percent))
                time.sleep(0.5)

            # 輸入完稅價格 (input#paid)
            paid_input = wait.until(EC.element_to_be_clickable((By.ID, "paid")))
            try:
                paid_input.clear()
            except Exception:
                driver.execute_script("arguments[0].value = '';", paid_input)
            human_type(paid_input, str(assessed_price))
            time.sleep(0.5)
            # 輸入數量 (input#taxCount2)
            taxCount2 = wait.until(EC.element_to_be_clickable((By.ID, "taxCount2")))
            try:
                taxCount2.clear()
            except Exception:
                driver.execute_script("arguments[0].value = '';", taxCount2)
            human_type(taxCount2, str(quantity))
            time.sleep(0.5)
            # 按下第二個「計算」按鈕
            calc_buttons = driver.find_elements(By.XPATH, "//button[@title='計算']")
            calc_buttons[1].click()
            time.sleep(2)
            # 讀取結果 (第二個 span.text-danger)
            result_spans = driver.find_elements(By.CSS_SELECTOR, "div.d-flex.align-items-center span.text-danger")
            if len(result_spans) >= 2:
                try:
                    online_result = Decimal(result_spans[1].text.strip())
                except Exception:
                    online_result = None
            else:
                online_result = None

            # 比對結果
            if online_result is None or Decimal(str(local_result)) != online_result:
                print("不一致測試案例 (從價稅):")
                print(f"big_category: {big_category}, sub_category: {sub_category}, quantity: {quantity}, assessed_price: {assessed_price}, is_electric: {is_electric}")
                print(f"本地計算: {local_result}, 網頁結果: {online_result}")
                mismatches += 1

        # 每次測試後刷新頁面以重置表單
        driver.refresh()
        time.sleep(2)

    print(f"總測試次數: {total_tests}, 不一致次數: {mismatches}")
    driver.quit()

if __name__ == '__main__':
    run_tax_calculator_test()


===== 測試案例 1 =====


TimeoutException: Message: 


### 隨機參數 直接手動作

In [ ]:
import math, random
from z3 import Real, Solver, sat

# ------------------------------
# 計算器 function（固定稅額）
# ------------------------------
def compute_tax_fixed(big_category, sub_category, quantity):
    """
    計算固定稅額類別的應納稅額（針對水泥及油氣類）
    計算公式: 應納稅額 = (單位稅額) * (數量)
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    if big_category == "cement":
        mapping = {
            "white": 600,         # 白水泥或有色水泥，每公噸600元
            "bute1": 320,          # 卜特蘭一型
            "bute2": 196,          # 卜特蘭高爐
            "other_cement": 440    # 代水泥及其他水泥
        }
    elif big_category == "oilgas":
        mapping = {
            "gasoline": 6830,      # 汽油，每公秉6830元
            "diesel": 3990,        # 柴油，每公秉3990元
            "kerosene": 4250,      # 煤油，每公秉4250元
            "aviation": 610,       # 航空燃油，每公秉610元
            "fuel_oil": 110,       # 燃料油，每公秉110元
            "solvent_oil": 720,    # 溶劑油，每公秉720元
            "lpg": 690             # 液化石油氣，每公噸690元
        }
    else:
        mapping = {}
    
    unit_tax = mapping.get(sub_category, 0)
    s.add(tax_amount == unit_tax * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

# ------------------------------
# 計算器 function（從價稅）
# ------------------------------
def compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric=False):
    """
    計算從價稅類別的應納稅額（針對橡膠輪胎、飲料品、平板玻璃、電器、車輛）
    計算公式: 應納稅額 = assessed_price * (從價稅率) * quantity
      若為車輛且 is_electric 為 True，則稅率折半
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    if big_category == "rubber":
        mapping = {"bus_truck": 0.10, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "beverage":
        mapping = {"diluted_juice": 0.08, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "flat_glass":
        mapping = {"regular": 0.10, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "electrical":
        mapping = {
            "refrigerator": 0.13,
            "color_tv": 0.13,
            "central_aircon": 0.15,
            "non_central_aircon": 0.20,
            "dehumidifier": 0.15,
            "videorecorder": 0.13,
            "non_portable_recordplayer": 0.10,
            "portable_recordplayer": 0.0,
            "tape_recorder": 0.10,
            "stereo": 0.10,
            "electric_oven": 0.15
        }
        rate = mapping.get(sub_category, 0)
    elif big_category == "vehicle":
        mapping = {
            "compact_car": 0.25,
            "large_car": 0.30,
            "truck_bus_others": 0.15,
            "motorcycle": 0.17
        }
        rate = mapping.get(sub_category, 0)
        if is_electric:
            rate = rate / 2.0
    else:
        rate = 0
    
    s.add(tax_amount == assessed_price * rate * quantity)
    
    if s.check() == sat:
        m = s.model()
        tax_value = int(float(m[tax_amount].as_decimal(20)))
        return (tax_value)
    else:
        return None
    
def round_half_up(x: float) -> int:
    """
    正常四捨五入函式：
      當小數部分 >= 0.5 時，往上取整，
      適用於非負數情形（稅額通常為正）。
    """
    return int(math.floor(x + 0.5))
# ------------------------------
# 隨機產生 100 組參數，並計算本地結果
# ------------------------------

# 固定稅額參數設定：
fixed_big_categories = {
    "cement": ["white", "bute1", "bute2", "other_cement"],
    "oilgas": ["gasoline", "diesel", "kerosene", "aviation", "fuel_oil", "solvent_oil", "lpg"]
}

# 從價稅參數設定：
others_big_categories = {
    "rubber": ["bus_truck", "other", "exempt"],
    "beverage": ["diluted_juice", "other", "exempt"],
    "flat_glass": ["regular", "exempt"],
    "electrical": ["refrigerator", "color_tv", "central_aircon", "non_central_aircon", 
                   "dehumidifier", "videorecorder", "non_portable_recordplayer", "portable_recordplayer",
                   "tape_recorder", "stereo", "electric_oven"],
    "vehicle": ["compact_car", "large_car", "truck_bus_others", "motorcycle"]
}

# 儲存測試資料
random_tests = []

for i in range(100):
    test_case = {}
    # 隨機決定使用固定稅額或從價稅 (0 固定, 1 從價)
    tax_type = random.choice([0, 1])
    test_case["tax_type"] = "fixed" if tax_type == 0 else "others"
    if tax_type == 0:
        # 固定稅額參數
        big_category = random.choice(list(fixed_big_categories.keys()))
        sub_category = random.choice(fixed_big_categories[big_category])
        quantity = random.randint(1, 50)
        test_case["big_category"] = big_category
        test_case["sub_category"] = sub_category
        test_case["quantity"] = quantity
        # 本地計算結果
        local_result = compute_tax_fixed(big_category, sub_category, quantity)
        test_case["local_result"] = local_result
    else:
        # 從價稅參數
        big_category = random.choice(list(others_big_categories.keys()))
        sub_category = random.choice(others_big_categories[big_category])
        quantity = random.randint(1, 50)
        assessed_price = random.randint(100, 1000000)
        is_electric = False
        if big_category == "vehicle":
            is_electric = random.choice([True, False])
        test_case["big_category"] = big_category
        test_case["sub_category"] = sub_category
        test_case["quantity"] = quantity
        test_case["assessed_price"] = assessed_price
        test_case["is_electric"] = is_electric
        # 本地計算結果
        local_result = compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric)
        test_case["local_result"] = local_result
    random_tests.append(test_case)

# 印出 100 組隨機參數與本地計算結果（轉換成浮點數顯示）
for idx, test in enumerate(random_tests, 1):
    # 如果結果為有理數，嘗試轉換成小數表示
    lr = test["local_result"]
    print(f"Test {idx}: {test}")


Test 1: {'tax_type': 'fixed', 'big_category': 'oilgas', 'sub_category': 'aviation', 'quantity': 27, 'local_result': 16470}
Test 2: {'tax_type': 'others', 'big_category': 'beverage', 'sub_category': 'other', 'quantity': 47, 'assessed_price': 26229, 'is_electric': False, 'local_result': 184914}
Test 3: {'tax_type': 'others', 'big_category': 'electrical', 'sub_category': 'non_portable_recordplayer', 'quantity': 27, 'assessed_price': 514698, 'is_electric': False, 'local_result': 1389684}
Test 4: {'tax_type': 'others', 'big_category': 'vehicle', 'sub_category': 'large_car', 'quantity': 33, 'assessed_price': 873714, 'is_electric': False, 'local_result': 8649768}
Test 5: {'tax_type': 'others', 'big_category': 'flat_glass', 'sub_category': 'regular', 'quantity': 50, 'assessed_price': 534948, 'is_electric': False, 'local_result': 2674740}
Test 6: {'tax_type': 'others', 'big_category': 'beverage', 'sub_category': 'diluted_juice', 'quantity': 33, 'assessed_price': 178863, 'is_electric': False, 'l

### 最佳化範例

### 範例 1：在特定生產配額下，最小化總稅額
情境說明：
假設公司每個月有「生產數量」的上限，但必須生產「至少」多少水泥/油品/車輛，才能符合市場基本需求。希望在滿足最低需求與不得超過生產上限的條件下，最小化需要繳納的總貨物稅。

可能的 Constraint
水泥最低需求：>= 200（公噸），且最高不能超過 500。
油品最低需求： >= 100（公秉），最高 300。
車輛最低需求：>= 5 台，最高 20。
目標函式：Minimize( cement_tax_total + oil_tax_total + vehicle_tax_total )
也就是把所有應納稅額加總後，讓 Solver 幫我們找出最小的解。

In [47]:
from z3 import *

o = Optimize()

o.add(cement_quantity >= 200, cement_quantity <= 500)
o.add(oil_quantity >= 100, oil_quantity <= 300)
o.add(vehicle_output_qty >= 5, vehicle_output_qty <= 20)
total_tax = cement_tax_total + oil_tax_total + vehicle_tax_total
handle_min = o.minimize(total_tax)

# [3] 做 check，將結果存到 result
result = o.check()

if result == sat:
    model = o.model()
    print("最佳化解 (Min Tax):", o.lower(handle_min))
    # 或列印各項數量與稅額
else:
    print("無可行解或未知。")

# -----------------------------------------------------------
# 使用 Optimize() 來做「最佳化」
# -----------------------------------------------------------
o = Optimize()

# -----------------------------------------------------------
# 1. 水泥(第7條) - 從量課稅 (每公噸)
# -----------------------------------------------------------
cement_type = Int("cement_type")
cement_quantity = Real("cement_quantity")   # 公噸
cement_tax_total = Real("cement_tax_total") # 水泥稅額

# 水泥的稅率(元/公噸)，根據 cement_type
#   1=白水泥(600)、2=卜特蘭一型(320)、3=高爐(280)、4=代水泥或其他(440)
tax_per_ton = If(
    cement_type == 1, 600,
    If(
        cement_type == 2, 320,
        If(
            cement_type == 3, 280,
            If(cement_type == 4, 440, 0)
        )
    )
)

# 水泥稅額 = 數量(公噸) × 單位稅(元/公噸)
o.add(cement_tax_total == cement_quantity * tax_per_ton)

# -----------------------------------------------------------
# 2. 油氣(第10條) - 從量課稅 (每公秉或公噸)
# -----------------------------------------------------------
oil_type = Int("oil_type")
oil_quantity = Real("oil_quantity")     # 公秉 (或公噸)
oil_tax_total = Real("oil_tax_total")

# 稅率 (依公秉或公噸)
#   1=汽油(6830)、2=柴油(3990)、3=煤油(4250)、4=航空燃油(610)、
#   5=燃料油(110)、6=溶劑油(720)、7=液化石油氣(690/公噸)
oil_tax_per_unit = If(
    oil_type == 1, 6830,
    If(
        oil_type == 2, 3990,
        If(
            oil_type == 3, 4250,
            If(
                oil_type == 4, 610,
                If(
                    oil_type == 5, 110,
                    If(
                        oil_type == 6, 720,
                        If(oil_type == 7, 690, 0)
                    )
                )
            )
        )
    )
)

# 油氣稅額 = 數量 × 單位稅
o.add(oil_tax_total == oil_quantity * oil_tax_per_unit)

# -----------------------------------------------------------
# 3. 車輛(第12條) - 從價課稅 (每單位完稅價格 × 稅率 × 數量)
# -----------------------------------------------------------
vehicle_type = Int("vehicle_type")
engine_cc = Int("engine_cc")
vehicle_custom_value = Real("vehicle_custom_value")   # 每台車的完稅價格
vehicle_output_qty = Real("vehicle_output_qty")       # 當月份出廠台數
vehicle_tax_total = Real("vehicle_tax_total")         # 車輛稅額

# 車輛的原始稅率(%)
#   vehicle_type=1 => 小客車 (依排氣量：<=2000cc=25%，>2000cc=30%)
#   vehicle_type=2 => 貨車/大客車/其他 => 15%
#   vehicle_type=3 => 機車 => 17%
base_vehicle_rate_percent = If(
    vehicle_type == 1,
    If(engine_cc <= 2000, 25, 30),
    If(
        vehicle_type == 2, 15,
        If(vehicle_type == 3, 17, 0)
    )
)

# 轉為小數
final_vehicle_rate_decimal = base_vehicle_rate_percent / 100.0

# 車輛稅額 = 每單位完稅價格 × 稅率 × 出廠數量
o.add(vehicle_tax_total == vehicle_custom_value * final_vehicle_rate_decimal * vehicle_output_qty)

# -----------------------------------------------------------
# (A) 先指定產品類型 (範例簡化寫死)
# -----------------------------------------------------------
#   水泥 => 用白水泥(1)
#   油品 => 用煤油(3)
#   車輛 => 小客車(1)，排氣量1800cc => 對應稅率25%
o.add(cement_type == 1)
o.add(oil_type == 3)
o.add(vehicle_type == 1)
o.add(engine_cc == 1800)

#   小客車的完稅價格(假設) => 800,000 元
o.add(vehicle_custom_value == 800000)

# -----------------------------------------------------------
# (B) 設定「生產數量」上下限 (配額需求)
# -----------------------------------------------------------
#   水泥： 200 <= cement_quantity <= 500
o.add(cement_quantity >= 200, cement_quantity <= 500)

#   油品： 100 <= oil_quantity <= 300
o.add(oil_quantity >= 100, oil_quantity <= 300)

#   車輛： 5 <= vehicle_output_qty <= 20
o.add(vehicle_output_qty >= 5, vehicle_output_qty <= 20)

# -----------------------------------------------------------
# 目標：最小化「總稅額」(水泥 + 油品 + 車輛)
# -----------------------------------------------------------
total_tax = cement_tax_total + oil_tax_total + vehicle_tax_total
handle_min = o.minimize(total_tax)

# 做 check
result = o.check()
if result == sat:
    model = o.model()

    # 取得各項變數
    c_qty = model[cement_quantity]
    o_qty = model[oil_quantity]
    v_qty = model[vehicle_output_qty]

    c_tax_val = model[cement_tax_total]
    o_tax_val = model[oil_tax_total]
    v_tax_val = model[vehicle_tax_total]
    optimal_tax = model.evaluate(total_tax)

    print("=== 最小化總稅額的解 ===")
    print(f"水泥量 = {c_qty},  稅額 = {c_tax_val}")
    print(f"汽油量 = {o_qty},  稅額 = {o_tax_val}")
    print(f"汽車量 = {v_qty},  稅額 = {v_tax_val}")
    print(f"總稅額 (Min) = {optimal_tax}")
    print(f"Z3 內部計算之目標值 = {o.lower(handle_min)}  (可能是同一數值)")
else:
    print("No solution or unknown (unsat/unknown).")



最佳化解 (Min Tax): -1*oo
=== 最小化總稅額的解 ===
水泥量 = 200,  稅額 = 120000
汽油量 = 100,  稅額 = 425000
汽車量 = 5,  稅額 = 1000000
總稅額 (Min) = 1545000
Z3 內部計算之目標值 = 1545000  (可能是同一數值)
